<a href="https://colab.research.google.com/github/tamaUdon/learning-llm/blob/main/gpt_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[ja,sentencepiece,torch] pandas

In [ ]:
from transformers import pipeline

In [ ]:
# 後続するテキストを予測するパイプラインを作成
generator = pipeline(
    "text-generation", model="abeja/gpt2-large-japanese"
)

# テキスト生成
text = "日本で一番高い山は"
outputs = generator(text)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


日本で一番高い山は富士山と聞いています。 昨今、ニュースで目にはしていたものの本当に高いものなのか理解に苦しむことがしばしばです。 先日、私の祖父が亡くなり、今年の春には私が父のお


###BERTを使う

In [ ]:
import pandas as pd

# マスクされたトークンを予測するpipelineを作成
fill_mask = pipeline(
    "fill-mask", model="cl-tohoku/bert-base-japanese-v3"
)
masked_text = "今日の映画は刺激的で面白かった。この映画は[MASK]。"
# masked_text = "日本の首都は[MASK]である"
# [MASK]部分を予測
outputs = fill_mask(masked_text)
# 上位三件をテーブルで表示
display(pd.DataFrame(outputs[:3]))

,score,token,token_str,sequence
0,0.683933,23845,素晴らしい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 素晴らしい 。
1,0.101234,24683,面白い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 面白い 。
2,0.048003,26840,楽しい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 楽しい 。


### T5 (エンコーダ・デコーダ) を使う

In [ ]:
t2t_generator = pipeline(
    "text2text-generation", model="retrieva-jp/t5-large-long"
)

# マスクされたスパンを予測
masked_text = "日本で通貨を発行しているのは、<extra_id_0>である。"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


日本銀行


In [ ]:
# モデルの語彙に日本銀行は存在しない
"日本銀行"in t2t_generator.tokenizer.vocab

False

### バイト対符号化

In [ ]:
# 単語とその頻度
word_freqs = {
    "たのしい":6,
    "たのしさ":2,
    "うつくしい":4,
    "うつくしさ":1,
}

# 語彙を文字で初期化
vocab = sorted(set([char for word in word_freqs for char in word]))
# 単語とその分割状態
splits = {word: [char for char in word] for word in word_freqs}

In [ ]:
from collections import Counter

def compute_most_frequent_pair(
    splits: dict[str, list[str]]
) -> tuple[str, str]:
  """
  最も頻度の高い隣接するサブワードの組を計算する
  """

  pair_freqs = Counter() # サブワードの組のカウンタ

  for word, freq in word_freqs.items(): # すべての単語を処理
    split = splits[word] # 現在の単語の分割状態を取得

    # 全ての隣接したサブワードの組を取得
    for i in range(len(split)-1):
      pair = (split[i], split[i + 1]) # ("た","の"), ("の","し"),...
      # サブワードの組の頻度に単語の頻度を加算
      pair_freqs[pair] += freq

  # カウンタから最も頻度の高いサブワードの組を取得
  pair,_ = pair_freqs.most_common(1)[0]
  return pair


def merge_pair(
    target_pair: tuple[str, str],
    splits: dict[str, list[str]]
) -> dict[str, list[str]]:
  """
  サブワードの組を結合する
  """

  l_str, r_str = target_pair
  for word in word_freqs: # 全ての単語を処理
    split = splits[word] # 現在の単語の分割状態を取得
    i = 0

    # 全ての隣接したサブワードの組を取得
    while i < len(split) -1:
      # サブワードの組が結合対象と一致したら結合
      if split[i] == l_str and split[i + 1] == r_str:
        split = split[:i] + [l_str + r_str] + split[i + 2 :]
      i += 1
    splits[word] = split # 現在の結合状態を更新
  return splits

In [ ]:
# 結合回数を9回としてバイト符号化の語彙を計算する

for step in range(9):
  # 最も頻度の高い隣接するサブワードの組を計算
  target_pair = compute_most_frequent_pair(splits)
  # サブワードの組を結合
  splits = merge_pair(target_pair, splits)
  # 語彙にサブワードの組を追加
  vocab.append(target_pair)

print(vocab)

['い', 'う', 'く', 'さ', 'し', 'た', 'つ', 'の', ('し', 'い'), ('た', 'の'), ('たの', 'しい'), ('う', 'つ'), ('うつ', 'く'), ('うつく', 'しい'), ('し', 'さ'), ('たの', 'しさ'), ('うつく', 'しさ')]
